In [ ]:
import kagglehub

kagglehub.login() #fe2b59006df9190854e38e64ea5b88fc

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import pandas as pd
import torch
from datasets import Dataset
from peft import prepare_model_for_int8_training, get_peft_model, LoraConfig, TaskType



# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model in 4-bit precision mode
model_name = "facebook/opt-350m"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    load_in_4bit=True,  # Load the model with quantized weights
    device_map="auto"   # Automatically distribute the model across available GPUs
)
# Freeze all parameters in the base model
for param in model.parameters():
    param.requires_grad = False

# Prepare model for 8-bit training
model = prepare_model_for_int8_training(model)
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")



# Assuming the dataset is in CSV format
import pandas as pd

# Load the dataset into a pandas DataFrame
# The file is typically named "spam.csv" in this dataset
df_sms = pd.read_csv(f"{path}/spam.csv", encoding="ISO-8859-1")

# Rename columns to match expected names
df_sms = df_sms.rename(columns={"v1": "label", "v2": "text"})  # 'v1' is label, 'v2' is the message

# Drop unnecessary columns if present
df_sms = df_sms[['text', 'label']]

# Ensure consistent label encoding: 'ham' -> 0, 'spam' -> 1
df_sms['label'] = df_sms['label'].map({'ham': 0, 'spam': 1})

# Convert to HuggingFace Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df_sms)

# Split the dataset into training and validation sets (90% train, 10% validation)
train_data = dataset.select([i for i in range(len(dataset)) if i % 10 != 0])  # 90% for training
val_data = dataset.select([i for i in range(len(dataset)) if i % 10 == 0])  # 10% for validation

# Tokenize the input sequences
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', max_length=128, truncation=True)

# Apply tokenization to the datasets
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Convert labels to a format suitable for classification (e.g., 0 for ham, 1 for spam)
def encode_labels(examples):
    examples['labels'] = [1 if label == 'spam' else 0 for label in examples['label']]
    return examples

train_data = train_data.map(encode_labels, batched=True)
val_data = val_data.map(encode_labels, batched=True)

# Set up QLoRA (Low-Rank Adaptation)
lora_config = LoraConfig(
    r=8,                          # Rank of the low-rank matrices
    lora_alpha=16,                 # Scaling factor
    lora_dropout=0.1,              # Dropout rate
    task_type=TaskType.SEQ_CLS     # Sequence classification task
)

# Apply QLoRA to the model
model = get_peft_model(model, lora_config)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results_sms_spam",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    fp16=False,
)
"""training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    fp16=False,  # Disable FP16 precision
)"""

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)

# Start Training
trainer.train()


ImportError: cannot import name 'prepare_model_for_int8_training' from 'peft' (c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\peft\__init__.py)

: 